In [1]:
# RUN THIS CELL: it loads some style files
from IPython.core.display import HTML, display, Math
with open( './style/custom.css', 'r' ) as f: html_style = f.read()
HTML( html_style )

# Test binomiale (una coda)

Un'urna contiene monete equilibrate e monete difettose. Le monete equilibrate hanno probabilità di successo $p=1/2$ le monete difettose hanno probabilità di successo $p=3/4$. Non conosciamo la frazione di monete difettose. Nel gergo dei <mark>test di ipotesi</mark> riassumeremo questa situazione scrivendo:

$H_0:\kern3.5ex p=1/2\quad$ <mark>ipotesi nulla</mark>

$H_A:\kern3ex p=3/4\quad$ <mark>ipotesi alternativa</mark>
 
In questo modo dichiariamo che $p=1/2$ è ciò che consideraimo *normale* o *sano* e che  $p=3/4$ è il tipo di deviazione dalla normalità o *patologia* che vogliamo rilevare.

Estraiamo una moneta dall'urna e, per decidere tra equilibrata o difettosa, facciamo il seguente test: la lanciamo $40$ volte e se il numero dei successi è $\ge k$ la dichiariamo difettosa. Stiamo descivendo una famiglia di test, uno per ogni scelta del parametro $k$. Questo parametro lo decide chi progetta il test cercando di minimizzare gli errori. Vogliamo per prima cosa come varia la probabilità di falsi positivi/negativi al variare di $k$. 

Il test è una variabile aleatoria $X$ a valori in $\{0,\dots,40\}$. Lo spazio campionario $\Omega$ è diviso in due parti: $H_A$ e $H_0$.  L'insieme $H_A$ contiene quegli $\omega$ che corrispondono a $40$ lanci fatti con una moneta difettosa mentre $H_0$ contiene quegli $\omega$ che corrispondono a lanci con una moneta equilibrata. 

Condizionando a $H_0$ otteniamo $X\sim {\rm B}(40,\ 1/2)$. Condizionando a $H_A$ otteniamo $X\sim {\rm B}(40,\ 3/4)$.

In [4]:
from scipy.stats import binom                             # lib. for statistical functions
from ipywidgets import interactive, FloatSlider, IntSlider   # lib. for interactive graphic
from bokeh.io import push_notebook, show, output_notebook # lib. for graphic output
from bokeh.plotting import figure
output_notebook()
options = dict(plot_height=250,plot_width=700,
               tools="pan,wheel_zoom,reset,save,crosshair,box_select")
def printred(s):
    display(HTML('<pre style="font: bold 12pt Courier, serif;color:#a02">' + s + '</pre>') )
def printgrn(s):
    display(HTML('<pre style="font: bold 12pt Courier, serif;color:#080">' + s + '</pre>') )

Loading BokehJS ...

In [7]:
n = 40; p0 = 1/2; k_max = int(3*n/4)                                # Set the initial values of the parameters
x = range(0,n+1)

plot = figure(title="PMF of X ~ B(n, p) and the interval [k, n].",  # Create an empty figure
              x_axis_label = "# successes", y_axis_label = "probability", 
              x_range=(0,n+1), y_range=(0,max(binom.pmf(x,n,p0)*1.2)), **options )               
plot.title.text_font="courier"
plot.title.text_font_size="14pt"


r1 = plot.vbar(x, top=[0 for i in x], bottom=0, width=.9, color='green', alpha=0.5) # Initialize barplot
r2 = plot.vbar(x, top=[0 for i in x], bottom=0, width=.9, color='red',   alpha=0.5) # Initialize barplot
show(plot, notebook_handle=True)                                                    # Show barplot
        
def f(k, p, p0):
    data = {'x':x, 'top':  binom.pmf(x, n, p) }
    x_rif = range(k,n+1)
    data2 = {'x':x_rif, 'top':  binom.pmf(x_rif, n, p) }
    r1.data_source.data = data
    r2.data_source.data = data2
    printred('T = [ k, n ]  zona di rifiuto  k = {}  n = {}'.format(k,n) )
    if p == p0: 
        printred('Pr( T | p = {:.2f}) = α = {:.3f} (falso positivo) ERRORE I tipo'.format(p,1-binom.cdf(k-1, n, p)))
    else: 
        printgrn('Pr(¬T | p = {:.2f}) = β = {:.3f} (falso negativo) ERRORE II tipo'.format(p,binom.cdf(k-1, n, p)))
    push_notebook()
    
w1 = interactive(lambda k,p: f(k, p, 0.5),
         k = IntSlider  (description="k", min=n/2, max=k_max, step=1, value=n), 
         p = FloatSlider(description="p", min=0.5, max=0.75, step=0.05, value=0.5))
display(w1);

interactive(children=(IntSlider(value=30, description='k', max=30, min=20), FloatSlider(value=0.5, description…

# Osservazione

$\alpha =\Pr (X\in T\,|\, H_0)$ si chiama <mark>significatività</mark> 

$\displaystyle\alpha =\sum_{i\in T}{n \choose i}\cdot 0.5^i \cdot 0.5^{n-i}$ nell'esempio

$\alpha$ misura la probabilità si un errore del <mark>I tipo</mark> (falso positivo)

$\beta = \Pr (X\notin T\,|\, H_A)$ non ha nome, ma $1-\beta$ si chiama <mark>potenza</mark>, nei test diagnostici la potenza si chiamava *sensitività*. 

$\displaystyle\beta =\sum_{i\notin T}{n \choose i}\, 0.75^i \, 0.25^{n-i}$ nell'esempio

$\beta$ misura la probabilità si un errore del <mark>II tipo</mark> (falso negativo).

Per calcolare $\alpha$ e $\beta$ bisogna usare opportune librerie.

In [11]:
from scipy.stats import binom 

# calcolo di α per i seguenti valori di n e k:
n = 40
k = 23

1-binom.cdf(k-1,n,0.5)

0.21479525392169307

In [10]:
# calcolo di β (per gli stessi valori di n e k):
binom.cdf(k-1,n,0.75)

0.004651468079689692

Per trovare il la soglia ottimale del test, calcoliamo 
$\alpha$ e $\beta$ per diversi valori di $k$.

Conviene automatizzare il calcolo con un *ciclo for*. Ogni linguaggio di programmazione ha la sua specifica sintassi per un tale ciclo. 

In [16]:
n=21
# calcolo di α per n come sopra e tutti i valori di k ∈ [0,n] :
for k in range(0,n+1):
    print( "k = ", k, "    α = ", 100 * (1-binom.cdf(k,n,0.5) ), "%" )

k =  0     α =  99.99995231628418 %
k =  1     α =  99.99895095825195 %
k =  2     α =  99.98893737792969 %
k =  3     α =  99.92551803588867 %
k =  4     α =  99.6401309967041 %
k =  5     α =  98.66981506347656 %
k =  6     α =  96.08230590820312 %
k =  7     α =  90.53764343261719 %
k =  8     α =  80.8344841003418 %
k =  9     α =  66.81880950927737 %
k =  10     α =  50.0 %
k =  11     α =  33.18119049072262 %
k =  12     α =  19.165515899658203 %
k =  13     α =  9.462356567382812 %
k =  14     α =  3.917694091796875 %
k =  15     α =  1.3301849365234375 %
k =  16     α =  0.35986900329589844 %
k =  17     α =  0.07448196411132812 %
k =  18     α =  0.0110626220703125 %
k =  19     α =  0.001049041748046875 %
k =  20     α =  4.76837158203125e-05 %
k =  21     α =  0.0 %


In [17]:
# calcolo di β per n come sopra e tutti i valori di k ∈ [0,n] :
for k in range(0,n+1):
    print( "k = ", k, "    β = ", round( 100 * binom.cdf(k-1,n,0.75), 0), "%" )

k =  0     β =  0.0 %
k =  1     β =  0.0 %
k =  2     β =  0.0 %
k =  3     β =  0.0 %
k =  4     β =  0.0 %
k =  5     β =  0.0 %
k =  6     β =  0.0 %
k =  7     β =  0.0 %
k =  8     β =  0.0 %
k =  9     β =  0.0 %
k =  10     β =  0.0 %
k =  11     β =  1.0 %
k =  12     β =  2.0 %
k =  13     β =  6.0 %
k =  14     β =  13.0 %
k =  15     β =  26.0 %
k =  16     β =  43.0 %
k =  17     β =  63.0 %
k =  18     β =  81.0 %
k =  19     β =  93.0 %
k =  20     β =  98.0 %
k =  21     β =  100.0 %
